## Packages

In [1]:
import os
from snowflake.snowpark import Session
import pandas as pd
from geoClassy import single as gCs
import yaml

## geoClassy preamble

In [2]:
# GEOCLASSY PREAMBLE
gCs.requisites()
GEOJSON_PATH='USA New York City neighborhood 20190128.geojson'

gCs.loadFile(GEOJSON_PATH)

Json module correctly imported
Shapely Geometry module correctly imported


# Snowflake credentials

In [3]:
# Load credentials from YAML file
with open("credentials.yml", "r") as keyholder:
    try:
        credentials=yaml.safe_load(keyholder)
    except yaml.YAMLError as exc:
        print(exc)

In [4]:
CONNECTION_PARAMETERS = {
"account": credentials['snowflake']['account'],
"user": credentials['snowflake']['user'],
"password": credentials['snowflake']['password'],
"database": credentials['snowflake']['database'],
"schema": credentials['snowflake']['schema'],
"warehouse": credentials['snowflake']['warehouse'],
}

## Snowpark preamble

In [5]:
my_session = Session.builder.configs(CONNECTION_PARAMETERS).create() 

## Main script

### Use warehouse

In [6]:
my_session.sql(f'''USE WAREHOUSE "Flakes";''').collect()

[Row(status='Statement executed successfully.')]

### Load table rows to Pandas DF

In [7]:
geotable = my_session.table('Poi').sort('Name')
geotable_df = geotable.to_pandas()

In [8]:
geotable_df.head()

,LAT,LON,NAME,NEIGHBORHOOD
0,40.706700,-74.009800,Crysler Building,None
1,40.748417,-73.985833,Empire State Building,None
2,40.741111,-73.989722,Flatiron Building,None


### Use geoClassy to find the neighborhood for each POI of the DF

In [9]:
geotable_df['NEIGHBORHOOD'] = geotable_df.apply(lambda row: gCs.getNames(pd.to_numeric(row.LAT), pd.to_numeric(row.LON)), axis=1)

### Create a temp table on Snowflake and insert DF rows

In [10]:
snowpark_df = my_session.write_pandas(geotable_df, "POI_temp", auto_create_table=True, table_type="temporary")

### Update from the temp table

In [11]:
my_session.sql(f'''UPDATE "POI" A
    SET A.NEIGHBORHOOD=B.NEIGHBORHOOD
    FROM "POI_temp" B
    WHERE A.Lat=B.Lat AND A.Lon=B.Lon;''').collect()

[Row(number of rows updated=3, number of multi-joined rows updated=0)]

In [12]:
snowpark_df.to_pandas()

,LAT,LON,NAME,NEIGHBORHOOD
0,40.706700,-74.009800,Crysler Building,Financial District
1,40.748417,-73.985833,Empire State Building,Midtown South
2,40.741111,-73.989722,Flatiron Building,Flatiron District
